In [1]:
import csv
import sys
import statistics

In [2]:
csv_file = open('../dataset/FilmesBrasileirosLancados.csv', 'r')
csv_reader = csv.reader(csv_file, delimiter=';')
next(csv_reader) # skip header

['"Ano de Lançamento"',
 '"Certificado de Produto Brasileiro (CPB)"',
 '"Título"',
 '"Direção"',
 '"Gênero"',
 '"Empresa Produtora Brasileira Majoritária"',
 '"UF"',
 '"Empresa Produtora Minoritária Brasileira"',
 '"UF"',
 '"Distribuidora"',
 '"Máximo de Salas"',
 '"Renda (R$)"',
 '"Público "']

## Filtrando campos necessários

In [3]:
data = []
for row in csv_reader:
    if row[0] != '':
        if row[11] != '"ND"' and row[12] != '"ND"': # Removing movies that don't have all information
            movie = {"Diretor": row[3].replace('"', ''),"Genero": row[4].replace('"', ''),"Distribuidora": row[9].replace('"', ''),"Renda": row[11].replace('"', '').replace(',', '.'),"Publico": row[12].replace('"', '')}
            data.append(movie)

In [4]:
print(len(data))

1532


## Conversão Atributos Nominais

Diretores

In [5]:
allDirctr = []
for r in data:
    allDirctr.append(r['Diretor'])

In [15]:
director_lst = []
director_name = []
for r in data:
    if r['Diretor'] not in director_name and allDirctr.count(r['Diretor']) > 1:
        director_name.append(r['Diretor'])
        director_lst.append(r['Diretor'])

In [16]:
len(director_lst)

277

In [17]:
for r in data:
    r['Diretor'] = director_lst.index(r['Diretor'])+1 if r['Diretor'] in director_lst else 0

Genêro

In [20]:
genre_lst = []
for r in data:
    if r['Genero'] not in genre_lst:
        genre_lst.append(r['Genero'])

In [21]:
len(genre_lst)

4

In [22]:
for r in data:
    r['Genero'] = genre_lst.index(r['Genero'])

Distribuidora

In [23]:
distrib_lst = []
for r in data:
    if r['Distribuidora'] not in distrib_lst:
        distrib_lst.append(r['Distribuidora'])

In [24]:
len(distrib_lst)

317

In [25]:
for r in data:
    r['Distribuidora'] = distrib_lst.index(r['Distribuidora'])

## Exportando resultados

In [26]:
csv_out = []
for r in data:
    row = []
    row.append(r['Diretor'])
    row.append(r['Genero'])
    row.append(r['Distribuidora'])
    row.append(r['Renda'])
    row.append(r['Publico'])
    csv_out.append(row)

In [ ]:
with open('../dataset/RegressionAnalysisPreProcessing.csv', 'w') as writeFile:
    writer = csv.writer(writeFile)
    writer.writerows(csv_out)